In [8]:
using Plots, Printf, ForwardDiff, DualNumbers

# Constants and initial settings
const α = 1.0
const N = 64
const T = 1.0
const x = range(-1.0, 1.0, N+1); x = vec(x[1:end-1])
const Δx = x[2] - x[1]
const Δt = Δx / 10

# Initial condition function
ρ0(x) = -(x .- 1) .* (x .+ 1)

# Saturation functions
m1(x) = x
m2(x) = -x + α

# Diffusion function and its derivative
U(x) = x^2
dU(x) = ForwardDiff.derivative(U, x)

# Potential function
V(x) = x^2

# Flux ξ_i function
function ξ(ρ)
    dUρ = x -> dU(ρ(x))
    return x -> dUρ(x) + V(x)
end

# Velocity function
function vel(p, q)
    ξp = ξ(p)
    ξq = ξ(q)
    return x -> -(ξp(x) - ξq(x)) / Δx
end

# Flux F function
function F(p, q)
    vel_pq = vel(p, q)
    return x -> m1(q(x)) * m2(p(x)) * max(vel_pq(x), 0) +
                m1(p(x)) * m2(q(x)) * min(vel_pq(x), 0)
end

# Newton argument H(ρ, ρn)
function H(ρ, ρn)
    Drift = similar(ρn)
    for i in 2:N
        Drift[i] = F(ρ[i], ρ[i-1])(x[i])
    end
    y = similar(ρ)
    for i in 1:N
        y[i] = -ρn[i] + ρ[i] + (Δt / Δx) * (Drift[i] - Drift[i+1])
    end
    return y
end

# Jacobian dH(ρ, ρn)
function dH(ρ, ρn)
    return ForwardDiff.jacobian(ρ -> H(ρ, ρn), ρ)
end

# Fixed point iteration function
function fixed_point(ρ)
    ρn = copy(ρ)  # Initialize ρn using initial condition
    tol = 1e-5
    max_iter = 1000
    iter = 0
    δ = ones(N)
    while maximum(abs.(δ)) > tol && iter < max_iter
        δ = dH(ρ, ρn) \ H(ρ, ρn)
        ρ -= δ
        iter += 1
    end
    if iter == max_iter
        println("Fixed point did not converge")
    end
    return ρ
end

# Initial condition ρ
ρ = ρ0.(x)

# Plotting function
function plot_ρ(x, ρ, t)
    plot(x, ρ, ylim=[0.0, 1.5],
         title=@sprintf("t = %0.3f", t),
         linewidth=2, thickness_scaling=1.5, label="")
end

# Main loop to compute ρ evolution over time
plots = []
for n = 0:ceil(Int, T / Δt)
    global ρ
    ρ = fixed_point(ρ)
    push!(plots, plot_ρ(x, ρ, n * Δt))
end

# Show the animation of ρ evolution
anim = @animate for p in plots
    plot(p)
end
gif(anim, "evolution_ρ.gif", fps=5)


LoadError: MethodError: objects of type ForwardDiff.Dual{ForwardDiff.Tag{var"#79#80"{Vector{Float64}}, Float64}, Float64, 11} are not callable
Maybe you forgot to use an operator such as [36m*, ^, %, / etc. [39m?